In [ ]:
#%matplotlib notebook
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (12, 9)

import scipy
import scipy.stats
import math
import numpy as np
import pandas as pd

#import ipywidgets
#from ipywidgets import interact

## Cut images with signal on borders

In [ ]:
CSV_FILE_PATH = "../xps/best.csv"

#WAVELET_LABEL = "Wavelets-n4-K-k-C1-s3-m3-kill"
#WAVELET_LABEL = "WT-ref-f3-s3"
#WAVELET_LABEL = "WT-t24-f3-s4"
#WAVELET_LABEL = "WT-t28-f3-s5"
WAVELET_LABEL = "WT-ref-s2-2-3-3"

TAILCUT_LABEL = "Tailcut-5-10-kill"

PART = 0       # 0 for gamma, 1 for protons
MIN_NPE = 0
ANGLE = 0.2

df = pd.read_csv(CSV_FILE_PATH)

ref = df[df.Type == 'Ref']
ref = ref[ref.Part == PART]

tc = df[df.Type == TAILCUT_LABEL]
tc = tc[tc.Part == PART]

wt = df[df.Type == WAVELET_LABEL]
wt = wt[wt.Part == PART]

tc_wt = pd.merge(tc, wt, on="Id", how="outer", suffixes=('_tc', '_wt'))  #.dropna(how='any')
df = pd.merge(tc_wt, ref, on="Id", how="outer")

df["delta_tc"] = np.fmod(((df['hPsi'] - df['hPsi_tc']) * 180. / np.pi), 90.)
df["delta_wt"] = np.fmod(((df['hPsi'] - df['hPsi_wt']) * 180. / np.pi), 90.)

df.delta_tc = abs(df.delta_tc)
df.delta_wt = abs(df.delta_wt)

In [ ]:
#df.groupby(["Type","Part","Success"]).count()
#df.groupby(["Type","Part"]).describe()

### NPE on cleaned image

### Tailcut

In [ ]:
fig, ax = plt.subplots()

df[abs(df.delta_tc) <= ANGLE]['peSum_tc'].plot.hist(alpha=0.2, bottom=0.1,
                                                    bins=np.logspace(0.1, 6., 200),
                                                    label="delta psi <= {}° ({})".format(ANGLE, len(df[abs(df.delta_tc) <= ANGLE])),
                                                    legend=True)
df[abs(df.delta_tc) >  ANGLE]['peSum_tc'].plot.hist(alpha=0.2, bottom=0.1,
                                                    bins=np.logspace(0.1, 6., 200),
                                                    label="delta psi > {}° ({})".format(ANGLE, len(df[abs(df.delta_tc) > ANGLE])),
                                                    legend=True)

ax.axvline(x=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)

ax.set_title("Tailcut (NPE > {})".format(MIN_NPE), fontsize=20)
ax.set_xlabel("NPE", fontsize=20)
ax.set_ylabel("Counts", fontsize=20)

ax.set_xscale('log')
ax.set_yscale('log')

ax.legend(prop={'size': 18})

### Wavelets

In [ ]:
fig, ax = plt.subplots()

df[abs(df.delta_wt) <= ANGLE]['peSum_wt'].plot.hist(alpha=0.2, bottom=0.1,
                                                    bins=np.logspace(0.1, 6., 200),
                                                    label="delta psi <= {}° ({})".format(ANGLE, len(df[abs(df.delta_wt) <= ANGLE])),
                                                    legend=True)
df[abs(df.delta_wt) >  ANGLE]['peSum_wt'].plot.hist(alpha=0.2, bottom=0.1,
                                                    bins=np.logspace(0.1, 6., 200),
                                                    label="delta psi > {}° ({})".format(ANGLE, len(df[abs(df.delta_wt) >  ANGLE])),
                                                    legend=True)

ax.axvline(x=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)

ax.set_title("Wavelets (NPE > {})".format(MIN_NPE), fontsize=20)
ax.set_xlabel("NPE", fontsize=20)
ax.set_ylabel("Counts", fontsize=20)

ax.set_xscale('log')
ax.set_yscale('log')

ax.legend(prop={'size': 18})

### Epsilon energy

In [ ]:
fig, ax = plt.subplots()

ax.plot(df[abs(df.delta_tc) >  ANGLE]['Denergy_tc'],
        df[abs(df.delta_tc) >  ANGLE]['peSum_tc'],
        '.b', alpha=0.3,
        label="delta psi > {}°".format(ANGLE))
ax.plot(df[abs(df.delta_tc) <= ANGLE]['Denergy_tc'],
        df[abs(df.delta_tc) <= ANGLE]['peSum_tc'],
        '.r', alpha=0.9,
        label="delta psi <= {}°".format(ANGLE))

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlim(0.01, 1.)

ax.set_xlabel("e_energy", fontsize=20)
ax.set_ylabel("npe", fontsize=20)

ax.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

In [ ]:
fig, ax = plt.subplots()

ax.plot(df[abs(df.delta_wt) >  ANGLE]['Denergy_wt'],
        df[abs(df.delta_wt) >  ANGLE]['peSum_wt'],
        '.b', alpha=0.3,
        label="delta psi > {}°".format(ANGLE))
ax.plot(df[abs(df.delta_wt) <= ANGLE]['Denergy_wt'],
        df[abs(df.delta_wt) <= ANGLE]['peSum_wt'],
        '.r', alpha=0.9,
        label="delta psi <= {}°".format(ANGLE))

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlim(0.01, 1.)

ax.set_xlabel("e_energy", fontsize=20)
ax.set_ylabel("npe", fontsize=20)

ax.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)